# 응답 속도 최적화 전략
RAG 챗봇의 응답 속도는 검색 방식과 설정에 따라 크게 달라질 수 있습니다.
이번 실습에서는 자주 쓰이는 최적화 전략들을 실험해봅니다.

## 1. 실습 목표
- 검색 속도에 영향을 주는 요소들을 이해합니다
- Top-K 조정, Chunk 수 제한, Embedding 캐싱 등을 실습합니다

## 2. 사전 준비: 데이터 로딩 및 벡터 DB 구성

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()

embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")
with open("kr_constitution_cleaned.txt", "r", encoding="utf-8") as f:
    text = f.read()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.create_documents([text])
faiss = FAISS.from_documents(docs, embedding)
retriever = faiss.as_retriever()

## 3. Top-K 설정에 따른 속도 비교

In [15]:
import time

query = "국회의 임기는 몇 년인가요?"
for k in [1, 3, 5, 50]:
    retriever.search_kwargs['k'] = k
    start = time.time()
    _ = retriever.get_relevant_documents(query)
    duration = time.time() - start
    print(f"Top-{k} 검색 소요 시간: {duration:.4f}초")

Top-1 검색 소요 시간: 0.0922초
Top-3 검색 소요 시간: 0.0171초
Top-5 검색 소요 시간: 0.0160초
Top-50 검색 소요 시간: 0.0169초


## 4. Chunk 수 제한을 통한 속도 최적화

In [16]:
# 텍스트 일부만 사용 (절반만)
docs_half = docs[:len(docs)//2]
faiss_half = FAISS.from_documents(docs_half, embedding)
retriever_half = faiss_half.as_retriever(search_kwargs={"k": 3})

start = time.time()
_ = retriever_half.get_relevant_documents(query)
print("절반 분량 인덱스 검색 소요 시간:", time.time() - start)

절반 분량 인덱스 검색 소요 시간: 0.014031171798706055


## 5. Embedding 캐싱 전략
- FAISS 인덱스를 저장하고, 다시 불러와 재사용하는 방식
- 저장과 불러오기 시 동일한 임베딩 모델을 사용해야 합니다.
- 파일 경로는 상대경로나 절대경로 모두 사용 가능하며, .save_local()은 폴더를 자동 생성합니다.
- Streamlit 앱 등에서는 이 로딩 코드를 @st.cache_resource 등과 함께 사용하면 초기화 시간까지 줄일 수 있습니다.

## 6. 정리
- Top-K 값이 클수록 검색 시간이 늘어남
- 전체 문서 수를 줄이면 빠르게 반응 가능
- 실제 운영 환경에서는 캐싱, 필터링, 조건 검색 등을 병행하여 성능을 최적화합니다.